In [1]:
import cv2 as cv
import numpy as np

In [41]:
class FrameProcessor:
    def __init__(self):
        pass
    
    def processing_loop(self, source, lth, hth, max_frame_num = -1,\
                        alternative_source="", save_to_file=""):
        i = 0
        results = []

        output_file = None
        
        #out = cv.VideoWriter('outpy.avi', cv.VideoWriter_fourcc('M','J','P','G'),
#                               30, (WINDX, WINDY))
#         out.write(canvas)
#         out.release()
        skip = False

        while (True):
            #if(skip == False):
            retval, frame = source.read()

            if (retval == False):
                print("Cannot read frame")
                
                if (alternative_source != ""):
                    print("Opening alternative source ", alternative_source)
                    source = cv.VideoCapture(alternative_source)
                    continue
                
                else:
                    print("Exiting loop")
                    break

            result = self.process_frame(frame, lth, hth)
            
            results.append(result)

            key = cv.waitKey(100) & 0xFF

            i += 1

            if (key == ord('q')):
                break

            if (key == 32):
                skip = not skip
                        
            if (max_frame_num != -1 and i >= max_frame_num):
                break

        return results
    
    def process_frame(self, frame, lth, hth):
        return 5

In [42]:
#############################################
# YOUR DEFAULT PARAMETERS BELOW
#############################################

lth, hth = (2, 79, 155), (255, 255, 255)#(2, 82, 134), (255, 255, 255)#(2, 95, 187), (13, 137, 238)

lth2, hth2 = (2, 39, 134), (255, 255, 255)

#############################################
# YOUR DEFAULT PARAMETERS ABOVE
#############################################

class ColorFilterTuning(FrameProcessor):
    def __init__(self):
        super().__init__()
        
        cv.namedWindow("color_filter_parameters")
                
        cv.createTrackbar('rl', 'color_filter_parameters', lth[0], 255, self.nothing)
        cv.createTrackbar('gl', 'color_filter_parameters', lth[1], 255, self.nothing)
        cv.createTrackbar('bl', 'color_filter_parameters', lth[2], 255, self.nothing)
        cv.createTrackbar('rh', 'color_filter_parameters', hth[0], 255, self.nothing)
        cv.createTrackbar('gh', 'color_filter_parameters', hth[1], 255, self.nothing)
        cv.createTrackbar('bh', 'color_filter_parameters', hth[2], 255, self.nothing)

    def nothing(self, inp):
        pass
    
    def process_frame(self, frame, lth, hth):
        #frame_cp = cv.medianBlur(frame, 3)
        frame_hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)                
        
        low_th =  (cv.getTrackbarPos('rl', 'color_filter_parameters'),
                   cv.getTrackbarPos('gl', 'color_filter_parameters'),
                   cv.getTrackbarPos('bl', 'color_filter_parameters'))
        
        high_th = (cv.getTrackbarPos('rh', 'color_filter_parameters'),
                   cv.getTrackbarPos('gh', 'color_filter_parameters'),
                   cv.getTrackbarPos('bh', 'color_filter_parameters'))
        
        mask = cv.inRange(frame_hsv, low_th, high_th)
        
        #cv.imshow("frame", frame)
        cv.imshow("color_filter_parameters", mask)
        
        return (low_th, high_th)

In [43]:
video_file = "data_videos_fingers.mov"

cam = cv.VideoCapture(video_file)

#print(cam)
# frame_offset = 100
# cam.set(1, frame_offset)

tuner = ColorFilterTuning()

colors = tuner.processing_loop(cam, lth, hth, max_frame_num = -1,\
            alternative_source=video_file)
lth, hth = colors[-1]

print("Color filter parameters: ", lth, hth)
cam.release()
cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(100)

Color filter parameters:  (2, 79, 155) (255, 255, 255)


-1

In [55]:
class FingersCounter(FrameProcessor):
    def __init__(self):
        super().__init__()

        cv.namedWindow("frame")
                
        cv.createTrackbar('lth', 'frame', 125, 255, self.nothing)
        cv.createTrackbar('hth', 'frame', 175, 255, self.nothing)
        #cv.createTrackbar('ker', 'frame', 3, 10, self.nothing)
        #cv.createTrackbar("area", 'frame', 0, 100, self.nothing)

    def nothing(self, inp):
        pass
    
    def fill_holes (self, img):
        (h, w) = img.shape

        before_area = img.sum ()

        img_enlarged = np.zeros ((h + 2, w + 2), np.uint8)
        img_enlarged [1:h+1, 1:w+1] = img

        img_enl_not = cv.bitwise_not (img_enlarged)
        th, im_th = cv.threshold (img_enl_not, 220, 255, cv.THRESH_BINARY_INV)

        im_floodfill = im_th.copy()

        h, w = im_th.shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)

        cv.floodFill(im_floodfill, mask, (0,0), 255)
        im_floodfill_inv = cv.bitwise_not(im_floodfill)
        im_out = im_th | im_floodfill_inv

        result = im_out [1:h-1, 1:w-1]

        #after_area = result.sum ()
        
        return result
    
    def get_mask(self, frame, lth, hth):
        frame_cp = cv.medianBlur(frame, 7)

        frame_hsv = cv.cvtColor(frame_cp, cv.COLOR_BGR2HSV)

        mask = cv.inRange(frame_hsv, lth, hth)

        kernel = np.ones((7, 7), np.uint8)

        mask_cp = cv.dilate(mask, kernel)
        #mask_cp = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
        mask_cp = self.fill_holes(mask_cp)

        connectivity = 4
        output = cv.connectedComponentsWithStats(mask_cp, connectivity, cv.CV_32S)
        num_labels = output[0]
        labels = output[1]
        stats = output[2]

        max_area = 1
        max_label = 1
            
        for i in range(1, num_labels):
            area = stats[i, cv.CC_STAT_AREA]
                
            if (area > max_area):
                max_area = area
                max_label = i
            
        for i in range(1, len(stats)):
            if (i != max_label):
                mask_cp[np.where(labels == i)] = 0

        t = stats[max_label, cv.CC_STAT_TOP]
        l = stats[max_label, cv.CC_STAT_LEFT]
        w = stats[max_label, cv.CC_STAT_WIDTH]
        h = stats[max_label, cv.CC_STAT_HEIGHT]

        cv.rectangle(mask_cp, (l, t), (l + w, t + h), 255, -1)

        return mask_cp

        
    
    def process_frame_old(self, frame, lth, hth):
        w, h, _ = frame.shape

        frame_cp = cv.resize(frame, (h // 2, w // 2))

        size = np.size(frame_cp)
        skel = np.zeros((w // 2, h // 2), np.uint8)

        mask = self.get_mask(frame_cp, lth, hth)

        frame_binarized = cv.inRange(cv.cvtColor(frame_cp, cv.COLOR_BGR2HSV), lth2, hth2)
        
        frame_bin_masked = cv.bitwise_and(frame_binarized, mask)
        frame_bin_masked = self.fill_holes(frame_bin_masked)
        frame_bin_masked = cv.medianBlur(frame_bin_masked, 3)

        frame_bin_masked_cp = np.copy(frame_bin_masked)

        element = cv.getStructuringElement(cv.MORPH_CROSS, (3,3))

        done = False

        while( not done):
            eroded = cv.erode(frame_bin_masked_cp, element)
            temp = cv.dilate(eroded, element)
            temp = cv.subtract(frame_bin_masked_cp, temp)
            skel = cv.bitwise_or(skel, temp)
            frame_bin_masked_cp = eroded.copy()

            zeros = size - cv.countNonZero(frame_bin_masked_cp)
            if zeros==size:
                done = True

        
        skel_3ch = cv.cvtColor(skel, cv.COLOR_GRAY2BGR)
        frame_bin_masked_3ch = cv.cvtColor(frame_bin_masked, cv.COLOR_GRAY2BGR)
        res = np.concatenate((frame_cp, skel_3ch), axis=1)

        cv.imshow('frame', res)

        return 3
    
    def process_frame(self, frame, lth, hth):
        w, h, _ = frame.shape

        frame_cp = cv.resize(frame, (h // 2, w // 2))

        mask = self.get_mask(frame_cp, lth, hth)

        frame_binarized = cv.inRange(cv.cvtColor(frame_cp, cv.COLOR_BGR2HSV), lth2, hth2)
        
        frame_bin_masked = cv.bitwise_and(frame_binarized, mask)
        frame_bin_masked = self.fill_holes(frame_bin_masked)
        frame_bin_masked = cv.medianBlur(frame_bin_masked, 3)

        frame_bin_masked_cp = np.copy(frame_bin_masked)

        contours, hierarchy = cv.findContours(frame_bin_masked_cp, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        contours = max(contours, key=lambda x: cv.contourArea(x))
        cv.drawContours(frame_cp, [contours], -1, (255,255,0), 2)

        hull = cv.convexHull(contours)
        cv.drawContours(frame_cp, [hull], -1, (0, 255, 255), 2)

        hull = cv.convexHull(contours, returnPoints=False)
        defects = cv.convexityDefects(contours, hull)

        if defects is not None:
            cnt = 0
        else:
            return 0
        for i in range(defects.shape[0]): 
            s, e, f, d = defects[i][0]
            start = tuple(contours[s][0])
            end = tuple(contours[e][0])
            far = tuple(contours[f][0])
            a = np.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = np.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = np.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))
            if angle <= np.pi / 2:  
                cnt += 1
                cv.circle(frame_cp, far, 4, [0, 0, 255], -1)
        if cnt > 0:
            cnt = cnt+1
            
        cv.imshow('final_result',frame_cp)

        return  cnt

In [56]:
cam = cv.VideoCapture(video_file)

finger_counter = FingersCounter()

fingers_num = finger_counter.processing_loop(cam, lth, hth, max_frame_num=-1, alternative_source=video_file)

print(fingers_num)

cam.release()
cv.waitKey(0)
cv.destroyAllWindows()
cv.waitKey(100)

[5, 5, 0, 0, 0, 4, 5, 6, 0, 0, 0, 6, 5, 5, 5, 4, 3, 3, 3, 3, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 3, 5, 5, 5, 6, 5, 5, 5, 5, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 3, 0, 0, 0, 0, 0, 2]


-1

In [52]:
reference_fingers_num = [5, 5, 1, 0, 0, 5, 5, 5, 0, 0, 0, 5, 5, 5, 5, 4, 3, 3,\
                         3, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2,\
                         2, 2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,\
                         2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,\
                         2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1,\
                         3, 4, 0, 0, 0, 1]

max_grade = 100

corr_num = 0

for r, s in zip(reference_fingers_num, fingers_num):
    if (r == s):
        corr_num += 1

acc = corr_num / len(reference_fingers_num)
print(acc)

#print("correct ", corr_num, " out of ", len(reference_fingers_num),
#      corr_num / len(reference_fingers_num))

grade = min(acc * 2, 1) * max_grade

print("Your grade is ", "\033[92m{}\033[0m".format(str(int(grade)) +\
        " out of " + str(max_grade) + "; " + str(corr_num) + " frames out of "
        + str(len(reference_fingers_num))))

0.5104166666666666
Your grade is  100 out of 100; 49 frames out of 96


In [1]:
!git status

Текущая ветка: main
Эта ветка соответствует «origin/main».

Изменения, которые не в индексе для коммита:
  (используйте «git add <файл>...», чтобы добавить файл в индекс)
  (используйте «git restore <файл>...», чтобы отменить изменения в рабочем каталоге)
	изменено:      github_hw1.ipynb
	изменено:      github_sem1.ipynb

Неотслеживаемые файлы:
  (используйте «git add <файл>...», чтобы добавить в то, что будет включено в коммит)
	cv_hw1.ipynb

индекс пуст (используйте «git add» и/или «git commit -a»)


In [ ]:
!git add cv_hw1.